In [1]:
import numpy as np
import pandas as pd
import mne
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

ЗАГРУЗКА И ПРЕПРОЦЕССИНГ

In [2]:
raw = mne.io.read_raw_nihon('./00000000/NKT/EEG2100/FA0183FS.EEG', preload=True)
sfreq = raw.info['sfreq']  # 500 Гц

# Создание биполярного монтажа (как в DotNetChecker)
anodes = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
          'Fp1', 'Fp2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6',
          'Fz', 'Cz']
cathodes = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
            'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'O1', 'O2',
            'Cz', 'Pz']
ch_names = [f'{a}-{c}' for a, c in zip(anodes, cathodes)]

raw_bip = mne.set_bipolar_reference(
    raw, anode=anodes, cathode=cathodes, 
    ch_name=ch_names, drop_refs=True, copy=True
)
raw_bip.pick(ch_names)  # Только нужные 18 каналов

# Получаем данные
data = raw_bip.get_data()  # [18, n_times]
print(f"Данные загружены: {data.shape}")
print(f"Частота дискретизации: {sfreq} Гц")

Loading FA0183FS.EEG
Found 21E file, reading channel names.
Reading header from /home/jovyan/vspyatochkin/diploma/00000000/NKT/EEG2100/FA0183FS.EEG
Reading 0 ... 19248999  =      0.000 ... 38497.998 secs...


/tmp/ipykernel_209050/1261963923.py:1: RuntimeWarning: No PNT file exists. Metadata will be blank
  raw = mne.io.read_raw_nihon('./00000000/NKT/EEG2100/FA0183FS.EEG', preload=True)


Found LOG file, reading events.
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=18, n_times=19249000
    Range : 0 ... 19248999 =      0.000 ... 38497.998 secs
Ready.
Added the following bipolar channels:
Fp1-F3, Fp2-F4, F3-C3, F4-C4, C3-P3, C4-P4, P3-O1, P4-O2, Fp1-F7, Fp2-F8, F7-T3, F8-T4, T3-T5, T4-T6, T5-O1, T6-O2, Fz-Cz, Cz-Pz
Данные загружены: (18, 19249000)
Частота дискретизации: 500.0 Гц


СОЗДАНИЕ МЕТОК ДЛЯ КЛАССИФИКАЦИИ

In [3]:
df = pd.read_csv('depd_intervals.csv')  # channel, start_seconds, end_seconds

# Размечены только первые 36 минут
ANNOTATED_DURATION = 36 * 60
annotated_samples = int(ANNOTATED_DURATION * sfreq)
data = data[:, :annotated_samples]  # [18, 1,080,000]
n_channels, n_times = data.shape

print(f"Данные для классификации: {data.shape}")
print(f"Длительность: {n_times/sfreq/60:.1f} минут")
print(f"Всего событий ДЭРД: {len(df)}")

# Создаем список событий для быстрого поиска
events = []
for _, row in df.iterrows():
    ch = int(row['channel']) - 1  # Каналы с 1 → с 0
    start_sample = int(row['start_seconds'] * sfreq)
    end_sample = int(row['end_seconds'] * sfreq)
    
    if 0 <= ch < n_channels and start_sample < n_times:
        end_sample = min(end_sample, n_times)
        events.append({
            'channel': ch,
            'start': start_sample,
            'end': end_sample,
            'duration': end_sample - start_sample
        })

print(f"Загружено {len(events)} событий")
if events:
    avg_duration = np.mean([e['duration'] for e in events]) / sfreq
    print(f"Средняя длительность ДЭРД: {avg_duration:.3f} сек")

Данные для классификации: (18, 1080000)
Длительность: 36.0 минут
Всего событий ДЭРД: 387
Загружено 387 событий
Средняя длительность ДЭРД: 0.326 сек


СОЗДАНИЕ ОКОН

In [4]:
def create_windows_for_classification(data, events, window_sec=2.0, overlap_sec=1.5):
    """Создает короткие окна для классификации"""
    window_samples = int(window_sec * sfreq)  # 1000 отсчетов (2 сек)
    step_samples = int((window_sec - overlap_sec) * sfreq)  # 250 отсчетов (0.5 сек)
    
    windows_data = []
    windows_labels = []  # Метки: [0/1 для каждого канала]
    
    n_samples = data.shape[1]
    
    # Создаем хэш-таблицу событий для быстрого поиска
    # Ключ: (start_sample, channel), Значение: end_sample
    event_dict = {}
    for event in events:
        key = (event['start'], event['channel'])
        event_dict[key] = event['end']
    
    for start in range(0, n_samples - window_samples + 1, step_samples):
        end = start + window_samples
        
        window_data = data[:, start:end]  # [18, 1000]
        
        # Метка: есть ли ДЭРД в этом окне для каждого канала
        label = np.zeros(n_channels, dtype=float)
        
        # Проверяем для каждого канала
        for ch in range(n_channels):
            # Ищем события на этом канале, которые пересекаются с окном
            has_depd = False
            for event_start, event_ch in event_dict.keys():
                if event_ch == ch:  # Событие на этом канале
                    event_end = event_dict[(event_start, ch)]
                    # Проверяем пересечение события с окном
                    if not (event_end <= start or event_start >= end):
                        has_depd = True
                        break
            
            label[ch] = 1.0 if has_depd else 0.0
        
        windows_data.append(window_data)
        windows_labels.append(label)
    
    return np.array(windows_data), np.array(windows_labels)

# Создаем окна 2 секунды
windows_data, windows_labels = create_windows_for_classification(
    data, events, window_sec=2.0, overlap_sec=1.5
)

print(f"\nСоздано окон: {len(windows_data)}")
print(f"Размер окна: {windows_data[0].shape}")
print(f"Процент окон с ДЭРД: {100 * windows_labels.sum() / windows_labels.size:.2f}%")
print(f"Окон с ДЭРД: {(windows_labels.sum(axis=1) > 0).sum()} из {len(windows_data)}")



Создано окон: 4317
Размер окна: (18, 1000)
Процент окон с ДЭРД: 1.99%
Окон с ДЭРД: 732 из 4317


БАЛАНСИРОВКА